In [8]:
import sqlite3
import pandas as pd
from pathlib import Path

# Connect to database (go up one level from notebooks/ folder)
db_path = Path("../data/geopolitical_monitor.db")
conn = sqlite3.connect(db_path)

# Query to match Muestra_Santiago.csv format
query = """
SELECT 
    a.news_id as "news id",
    a.news_title as "news title",
    SUBSTR(a.news_text, 1, 500) as "news text",
    a.article_summary as "article summary",
    e.event_id as "event id",
    e.event_summary as "event summary",
    a.publication_date as "publication date",
    e.event_date as "event date",
    e.event_location as "event location",
    e.dimension,
    e.event_type as "event type",
    e.sub_dimension as "sub dimension",
    (
        SELECT GROUP_CONCAT(DISTINCT ea.actor_iso3)
        FROM event_actors ea 
        WHERE ea.event_id = e.event_id
    ) as "actor list",
    (
        SELECT GROUP_CONCAT(ea.actor_iso3)
        FROM event_actors ea 
        WHERE ea.event_id = e.event_id AND ea.actor_role = 'actor1'
    ) as "actor1",
    (
        SELECT GROUP_CONCAT(ea.actor_iso3)
        FROM event_actors ea 
        WHERE ea.event_id = e.event_id AND ea.actor_role = 'actor1_secondary'
    ) as "actor1 secondary",
    (
        SELECT GROUP_CONCAT(ea.actor_iso3)
        FROM event_actors ea 
        WHERE ea.event_id = e.event_id AND ea.actor_role = 'actor2'
    ) as "actor2",
    (
        SELECT GROUP_CONCAT(ea.actor_iso3)
        FROM event_actors ea 
        WHERE ea.event_id = e.event_id AND ea.actor_role = 'actor2_secondary'
    ) as "actor2 secondary",
    e.direction,
    e.sentiment
FROM events e
JOIN articles a ON e.news_id = a.news_id
ORDER BY a.news_id, e.event_id
"""

# Load into DataFrame
df = pd.read_sql_query(query, conn)

# Display settings for better viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

# Show the data
print(f"Total events: {len(df)}")
df.head()

Total events: 27


,news id,news title,news text,article summary,event id,event summary,publication date,event date,event location,dimension,event type,sub dimension,actor list,actor1,actor1 secondary,actor2,actor2 secondary,direction,sentiment
0,2,Trump to pardon ex-Honduras president convicted of drug trafficking,Trump to pardon ex-Honduras president convicted of drug trafficking\n\nJuan Orlando Hernández wa...,,2-1,"Trump announced a pardon for former Honduran President Juan Orlando Hernández, who was convicted...",2025-11-29T16:32:00,2025-11-29,Honduras,Political Relations,pardon,diplomatic,"USA,HND",USA,None,HND,None,unilateral,5.0
1,2,Trump to pardon ex-Honduras president convicted of drug trafficking,Trump to pardon ex-Honduras president convicted of drug trafficking\n\nJuan Orlando Hernández wa...,,2-2,"Trump endorsed Nasry Asfura for the Honduran presidency, criticizing other candidates and emphas...",2025-11-29T16:32:00,2025-11-29,Honduras,Political Relations,endorsement,diplomatic,"USA,HND,VEN",USA,None,HND,VEN,bilateral,3.0
2,4,South Africa hits back after Trump says US won't invite it for G20 next year,South Africa hits back after Trump says US won't invite it for G20 next year\n\nRamaphosa said t...,,4-1,South Africa's President Ramaphosa responded to Trump's announcement that South Africa would not...,2025-11-27T14:24:55,2025-11-27,USA,Political Relations,diplomatic meeting,diplomatic,"USA,ZAF",USA,None,ZAF,None,unilateral,-5.0
3,5,How Trump's pledge to tackle Sudan atrocities could play out,How Trump's pledge to tackle Sudan atrocities could play out\n\n16 hours ago Share Save Alex de ...,,5-1,"President Trump pledges to intervene in the Sudan conflict, collaborating with Egypt, Saudi Arab...",2025-11-29T01:28:21,2025-11-29,Sudan,Political Relations,diplomatic meeting,diplomatic,"USA,SDN,ARE,SAU,EGY",USA,None,SDN,"ARE,SAU,EGY",bilateral,5.0
4,6,Trump plans to pardon former Honduran President Juan Orlando Hernandez,Trump plans to pardon former Honduran President Juan Orlando Hernandez\n\ntoggle caption Elmer M...,,6-1,"President Trump plans to pardon former Honduran President Juan Orlando Hernandez, which may infl...",2025-11-29T00:19:02,2025-11-29,Honduras,Political Relations,pardon,diplomatic,"USA,HND",USA,None,HND,None,unilateral,5.0


In [6]:
# Export to CSV (same format as Muestra_Santiago.csv)
df.to_csv("../data/events_export.csv", index=False)
print("✅ Exported to data/events_export.csv")

✅ Exported to data/events_export.csv


In [7]:
# Quick summary statistics
print("\n📊 Summary Statistics")
print("=" * 50)
print(f"Total Articles: {df['news id'].nunique()}")
print(f"Total Events: {len(df)}")
print(f"Avg Events per Article: {len(df) / df['news id'].nunique():.1f}")
print(f"\nSentiment: {df['sentiment'].mean():.1f} avg ({df['sentiment'].min():.0f} to {df['sentiment'].max():.0f})")
print(f"\nBy Dimension:")
print(df['dimension'].value_counts().to_string())
print(f"\nBy Direction:")
print(df['direction'].value_counts().to_string())


📊 Summary Statistics
Total Articles: 20
Total Events: 27
Avg Events per Article: 1.4

Sentiment: -2.0 avg (-7 to 5)

By Dimension:
dimension
Political Relations    19
Material Conflict       4
Economic Relations      4

By Direction:
direction
unilateral    15
bilateral     12


In [2]:
from pathlib import Path
import sys
# Ensure project root is on path (works from notebooks/ or project root)
_root = Path.cwd().parent if (Path.cwd() / ".." / "config").resolve().exists() else Path.cwd()
sys.path.insert(0, str(_root.resolve()))
from config.settings import get_settings
from src.data.database import set_db_path, set_database_url, get_db_connection, _execute_query, _row_to_dict

s = get_settings()
if s.database_url:
    set_database_url(s.database_url)
else:
    set_db_path(s.db_path)

with get_db_connection() as conn:
    c = _execute_query(conn, 'SELECT news_id, news_title, source_country, language_detected, LENGTH(news_text) as text_len, publication_date, source_domain FROM articles ORDER BY news_id DESC LIMIT 15')
    rows = c.fetchall()
    print('Articles (newest first, max 15):')
    print('-' * 90)
    for r in rows:
        d = _row_to_dict(r, c)
        print(f"news_id={d.get('news_id')}")
        print(f"  title: {(d.get('news_title') or '')[:70]}...")
        print(f"  source_country: {d.get('source_country') or '(none)'}  |  language_detected: {d.get('language_detected') or '(none)'}")
        print(f"  text_len: {d.get('text_len')}  |  pub_date: {d.get('publication_date') or ''}  |  domain: {d.get('source_domain') or ''}")
        print()

Articles (newest first, max 15):
------------------------------------------------------------------------------------------
news_id=86
  title: Afghan border minister holds phone talks with Iran’s deputy foreign mi...
  source_country: Afghanistan  |  language_detected: en
  text_len: 677  |  pub_date: 2026-02-09T09:15:06  |  domain: Ariana News | Afghanistan News

news_id=85
  title: Terrorist activities observed along Afghanistan borders, says Lavrov...
  source_country: Afghanistan  |  language_detected: en
  text_len: 812  |  pub_date: 2026-02-09T10:05:07  |  domain: Ariana News | Afghanistan News

news_id=84
  title: Pakistan to repatriate nearly 20,000 Afghans awaiting US resettlement...
  source_country: Afghanistan  |  language_detected: en
  text_len: 1172  |  pub_date: 2026-02-09T14:05:54  |  domain: Ariana News | Afghanistan News

news_id=83
  title: Mexico, US agree to begin formal USMCA trade talks. Where does that le...
  source_country: (none)  |  language_detected: (non

In [ ]:
# Get the ENTIRE news text for article 84 (full copy-paste)
from pathlib import Path
import sys
_root = Path.cwd().parent if (Path.cwd() / ".." / "config").resolve().exists() else Path.cwd()
sys.path.insert(0, str(_root.resolve()))
from config.settings import get_settings
from src.data.database import set_db_path, set_database_url, get_db_connection, _execute_query, _row_to_dict

s = get_settings()
if s.database_url:
    set_database_url(s.database_url)
else:
    set_db_path(s.db_path)

with get_db_connection() as conn:
    c = _execute_query(conn, "SELECT news_id, news_title, news_text, source_country, language_detected, publication_date FROM articles WHERE news_id = 84")
    row = c.fetchone()
d = _row_to_dict(row, c) if row else {}
full_text = d.get("news_text") or ""
print(f"news_id: {d.get('news_id')}")
print(f"title: {d.get('news_title')}")
print(f"source_country: {d.get('source_country')} | language_detected: {d.get('language_detected')} | pub_date: {d.get('publication_date')}")
print(f"text length: {len(full_text)} chars")
print()
print("--- Full article text ---")
print(full_text)